In [1]:
#from __future__ import absolute_import, division, print_function

#!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

In [2]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
random.seed(42)

#from google.colab import drive
#drive.mount('/content/gdrive')

#filePath ="/content/gdrive/My Drive/ML_TeamProject/PS_20174392719_1491204439457_log.csv"
#filePath ="/content/gdrive/My Drive/PS_20174392719_1491204439457_log.csv"
filePath ="PS_20174392719_1491204439457_log.csv"

import pandas as pd
data = pd.read_csv(filePath, sep=",", index_col=0)

df = data.sample(n=950000, random_state=random.seed(42))

print(df.info())

C:\Users\giris\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 950000 entries, 138 to 542
Data columns (total 10 columns):
type              950000 non-null object
amount            950000 non-null float64
nameOrig          950000 non-null object
oldbalanceOrg     950000 non-null float64
newbalanceOrig    950000 non-null float64
nameDest          950000 non-null object
oldbalanceDest    950000 non-null float64
newbalanceDest    950000 non-null float64
isFraud           950000 non-null int64
isFlaggedFraud    950000 non-null int64
dtypes: float64(5), int64(2), object(3)
memory usage: 79.7+ MB
None


In [3]:
df.loc[df.isFraud == 1]


,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
step,,,,,,,,,,
709,TRANSFER,242296.17,C535302480,242296.17,0.00,C1442778560,0.00,0.00,1,0
534,TRANSFER,209340.53,C447114205,209340.53,0.00,C1658435419,0.00,0.00,1,0
36,CASH_OUT,32382.96,C1507237385,32382.96,0.00,C53062849,0.00,32382.96,1,0
115,TRANSFER,530377.28,C1741821297,530377.28,0.00,C182731485,0.00,0.00,1,0
278,TRANSFER,240580.40,C338881900,240580.40,0.00,C738144053,0.00,0.00,1,0
724,TRANSFER,1379501.18,C1083455984,1379501.18,0.00,C662399585,0.00,0.00,1,0
569,TRANSFER,51416.07,C1275321415,51416.07,0.00,C1417015196,0.00,0.00,1,0
450,TRANSFER,422261.63,C105750198,422261.63,0.00,C1172379830,0.00,0.00,1,0
680,TRANSFER,1685692.11,C1039970092,1685692.11,0.00,C476797859,0.00,0.00,1,0


In [4]:
df['IS_TRANSFER'] = 0

df['IS_CASH_OUT'] = 0

In [5]:
df.loc[df['type']=='TRANSFER', 'IS_TRANSFER'] =1

In [6]:
df.loc[df['type']=='CASH_OUT', 'IS_CASH_OUT'] =1

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950000 entries, 138 to 542
Data columns (total 12 columns):
type              950000 non-null object
amount            950000 non-null float64
nameOrig          950000 non-null object
oldbalanceOrg     950000 non-null float64
newbalanceOrig    950000 non-null float64
nameDest          950000 non-null object
oldbalanceDest    950000 non-null float64
newbalanceDest    950000 non-null float64
isFraud           950000 non-null int64
isFlaggedFraud    950000 non-null int64
IS_TRANSFER       950000 non-null int64
IS_CASH_OUT       950000 non-null int64
dtypes: float64(5), int64(4), object(3)
memory usage: 94.2+ MB


In [8]:
df_back_up = df

In [9]:
df[df['isFraud']==1].shape

(1223, 12)

In [10]:
y = df['isFraud'].values

In [11]:
y.shape

(950000,)

In [12]:
num_zeros = (y == 0).sum()
num_ones = (y == 1).sum()
num_zeros


948777

In [13]:
num_ones

1223

In [14]:
from sklearn import preprocessing

min_max_scaler1 = preprocessing.MinMaxScaler()
np_scaled1 = min_max_scaler1.fit_transform(df[['amount']])
amount_df_normalized = pd.DataFrame(np_scaled1)
amount_df_normalized.values

min_max_scaler2 = preprocessing.MinMaxScaler()
np_scaled2 = min_max_scaler2.fit_transform(df[['oldbalanceOrg']])
oldbalanceOrg_df_normalized = pd.DataFrame(np_scaled2)
oldbalanceOrg_df_normalized.values

min_max_scaler3 = preprocessing.MinMaxScaler()
np_scaled3 = min_max_scaler3.fit_transform(df[['newbalanceOrig']])
newbalanceOrig_df_normalized = pd.DataFrame(np_scaled3)
newbalanceOrig_df_normalized.values

min_max_scaler4 = preprocessing.MinMaxScaler()
np_scaled4 = min_max_scaler4.fit_transform(df[['oldbalanceDest']])
oldbalanceDest_df_normalized = pd.DataFrame(np_scaled4)
oldbalanceDest_df_normalized.values

min_max_scaler5 = preprocessing.MinMaxScaler()
np_scaled5 = min_max_scaler5.fit_transform(df[['newbalanceDest']])
newbalanceDest_df_normalized = pd.DataFrame(np_scaled5)
newbalanceDest_df_normalized.values
X_df = df.drop(['nameOrig', 'nameDest','isFraud','type'], axis=1)

In [15]:



X_df['amount']= amount_df_normalized.values

X_df['oldbalanceOrg'] = oldbalanceOrg_df_normalized.values

X_df['newbalanceOrig'] = newbalanceOrig_df_normalized.values

X_df['oldbalanceDest'] = oldbalanceDest_df_normalized.values

X_df['newbalanceDest'] = newbalanceDest_df_normalized.values

In [16]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950000 entries, 138 to 542
Data columns (total 8 columns):
amount            950000 non-null float64
oldbalanceOrg     950000 non-null float64
newbalanceOrig    950000 non-null float64
oldbalanceDest    950000 non-null float64
newbalanceDest    950000 non-null float64
isFlaggedFraud    950000 non-null int64
IS_TRANSFER       950000 non-null int64
IS_CASH_OUT       950000 non-null int64
dtypes: float64(5), int64(3)
memory usage: 65.2 MB


In [17]:
X = X_df.to_numpy()

In [18]:
X_df['amount']

step
138    0.003628
33     0.000239
372    0.010331
329    0.013494
382    0.000076
260    0.002638
255    0.000220
206    0.003911
180    0.005014
157    0.004802
14     0.000193
370    0.000103
214    0.001882
40     0.000242
276    0.000178
353    0.000466
329    0.001603
163    0.000419
259    0.000061
185    0.001063
300    0.001176
402    0.004510
183    0.000094
306    0.003806
17     0.000136
329    0.000180
307    0.000071
208    0.000604
403    0.000760
347    0.000018
         ...   
134    0.002206
225    0.000050
226    0.000084
283    0.000255
210    0.000532
305    0.001335
375    0.006341
13     0.002717
397    0.000003
156    0.003521
187    0.000104
277    0.000104
19     0.000697
276    0.002080
133    0.002073
164    0.001352
396    0.003386
347    0.001380
321    0.003338
281    0.010349
346    0.000321
331    0.005282
307    0.002413
15     0.003091
278    0.002294
399    0.000023
201    0.000042
17     0.000067
186    0.000194
542    0.000207
Name: amount, Lengt

In [19]:
#from collections import Counter
#from imblearn.over_sampling import SMOTE, ADASYN

#X_res, y_res = SMOTE().fit_resample(X, y)

#print('Original dataset y shape %s' % Counter(y))
#print('Resampled dataset shape y_res %s' % Counter(y_res))


from collections import Counter
from imblearn.over_sampling import RandomOverSampler
print('Original dataset shape %s' % Counter(y))
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))


Using TensorFlow backend.


Original dataset shape Counter({0: 948777, 1: 1223})
Resampled dataset shape Counter({0: 948777, 1: 948777})


In [20]:
from sklearn.model_selection import train_test_split

X = X_res
y = y_res

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.2)

In [21]:
X_train

array([[3.17563264e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [9.13351886e-02, 1.88084567e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.41006687e-02, 2.90372002e-02, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [1.00324606e-04, 2.06596278e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [8.76839883e-03, 1.80565730e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.55661275e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [22]:
X_test

array([[4.31917106e-05, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.09375331e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.02760971e-02, 2.11613433e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.50226299e-02, 3.09357749e-02, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [3.07874260e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [3.86122565e-05, 8.93251081e-05, 1.21123539e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [23]:
y_train

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [24]:
num_zeros_y_train = (y_test == 0).sum()
num_ones_y_train = (y_test == 1).sum()
num_zeros_y_train

189470

In [25]:
num_ones_y_train

190041

In [26]:
num_zeros_y_test = (y_test == 0).sum()
num_ones_y_test = (y_test == 1).sum()
num_zeros_y_test

189470

In [27]:
num_ones_y_test

190041

In [28]:
uniqueValues_y_train = np.unique(y_train)
uniqueValues_y_test = np.unique(y_test)
 
print('Unique Values : ',uniqueValues_y_train)

Unique Values :  [0 1]


In [29]:
print('Unique Values : ',uniqueValues_y_test)

Unique Values :  [0 1]


In [30]:
from tensorflow import keras

inputs = keras.Input(shape=(8,))
inputs.shape

TensorShape([Dimension(None), Dimension(8)])

In [31]:
from tensorflow.keras import layers
dense = layers.Dense(8, activation='relu')
xi = dense(inputs)

W0811 16:17:37.225687 24364 deprecation.py:506] From C:\Users\giris\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [32]:
x1 = layers.Dense(15, activation='relu')(xi)
x2 = layers.Dense(13, activation='relu')(x1)
x3 = layers.Dense(11, activation='relu')(x2)
x4 = layers.Dense(7, activation='relu')(x3)
x5 = layers.Dense(4, activation='relu')(x4)
x6 = layers.Dense(2, activation='relu')(x5)

In [33]:
outputs = layers.Dense(1, activation='softmax')(x6)

In [34]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [35]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 15)                135       
_________________________________________________________________
dense_2 (Dense)              (None, 13)                208       
_________________________________________________________________
dense_3 (Dense)              (None, 11)                154       
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 84        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 32    

In [36]:
keras.utils.plot_model(model, 'my_first_model_with_shape_info.png', show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [37]:
model.compile(loss='binary_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])

In [38]:
history = model.fit(X_train, y_train,
                    batch_size=10000,
                    epochs=20,
                    validation_split=0.2)

Train on 1214434 samples, validate on 303609 samples


W0811 16:17:38.472365 24364 deprecation.py:323] From C:\Users\giris\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 2/20
1214434/1214434 [==============================] - 5s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 3/20
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 4/20
1214434/1214434 [==============================] - 3s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 5/20
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 6/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 7/20
1214434/1214434 [==============================] - 3s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 

In [39]:
model.compile(loss='binary_crossentropy',
              optimizer='Adagrad',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=10000,
                    epochs=20,
                    validation_split=0.2)

Train on 1214434 samples, validate on 303609 samples


W0811 16:19:04.990303 24364 deprecation.py:506] From C:\Users\giris\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\adagrad.py:105: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 2/20
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 3/20
1214434/1214434 [==============================] - 3s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 4/20
1214434/1214434 [==============================] - 5s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 5/20
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 6/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 7/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 

In [40]:
model.compile(loss='binary_crossentropy',
              optimizer='Adamax',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=10000,
                    epochs=20,
                    validation_split=0.2)

Train on 1214434 samples, validate on 303609 samples
Epoch 1/20
1214434/1214434 [==============================] - 5s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 2/20
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 3/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 4/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 5/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 6/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 7/20
1214434/1214434 [==============================] - 3s 3us/sample - loss: 7.6720 - ac

In [41]:
model.compile(loss='binary_crossentropy',
              optimizer='Nadam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=10000,
                    epochs=20,
                    validation_split=0.2)

Train on 1214434 samples, validate on 303609 samples
Epoch 1/20
1214434/1214434 [==============================] - 5s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 2/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 3/20
1214434/1214434 [==============================] - 5s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 4/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 5/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 6/20
1214434/1214434 [==============================] - 3s 2us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 7/20
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - ac

In [42]:
model.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=10000,
                    epochs=7,
                    validation_split=0.2)

Train on 1214434 samples, validate on 303609 samples
Epoch 1/7
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 2/7
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 3/7
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 4/7
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 5/7
1214434/1214434 [==============================] - 4s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 6/7
1214434/1214434 [==============================] - 3s 3us/sample - loss: 7.6720 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 7/7
1214434/1214434 [==============================] - 4s 4us/sample - loss: 7.6720 - acc: 0.49

In [43]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])


history = model.fit(X_train, y_train,
                    batch_size=1000,
                    epochs=7, shuffle=True,
                    validation_split=0.2)


Train on 1214434 samples, validate on 303609 samples
Epoch 1/7
1214434/1214434 [==============================] - 12s 10us/sample - loss: 7.6721 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 2/7
1214434/1214434 [==============================] - 13s 11us/sample - loss: 7.6721 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 3/7
1214434/1214434 [==============================] - 13s 11us/sample - loss: 7.6721 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 4/7
1214434/1214434 [==============================] - 12s 10us/sample - loss: 7.6721 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 5/7
1214434/1214434 [==============================] - 12s 10us/sample - loss: 7.6721 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 6/7
1214434/1214434 [==============================] - 11s 9us/sample - loss: 7.6721 - acc: 0.4996 - val_loss: 7.6593 - val_acc: 0.5005
Epoch 7/7
1214434/1214434 [==============================] - 13s 11us/sample - loss: 7.672

In [44]:
test_scores = model.evaluate(X_test, y_test, verbose=0)

print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1]*100,'%')



Test loss: 7.655084534073066
Test accuracy: 50.07522702217102 %
